In [1]:
import pandas as pd
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langsmith import traceable
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]

In [3]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.1
)

In [4]:
# llm.invoke("What is the capital of France?")

In [5]:
from typing import Optional
from pydantic import BaseModel, Field

class AgentState(BaseModel):
    user_query: str = Field(..., description="Original User Question")
    explanation: Optional[str] = Field(None, description="Detailed Answer to the User Question")
    examples: Optional[str] = Field(None, description="Relevant Examples Illustrating the Answer")
    final_answer: Optional[str] = Field(None, description="Concise Final Answer for the User")

### Node 1 : Explanation

In [6]:
@traceable(name="Explain Node")
def explain_concept(state: AgentState):
    prompt = f"""
    Explain the following topic in simple beginner-friendly terms:

    Topic: {state.user_query}
    """
    response = llm.invoke(prompt)

    return {"explanation": response.content}


### Node 2 - Example

In [7]:
@traceable(name="Example Node")
def generate_examples(state: AgentState):
    prompt = f"""
    Based on this explanation:

    {state.explanation}

    Provide 3 real-world examples.
    """
    response = llm.invoke(prompt)

    return {"examples": response.content}


### Node 3 — Summary Step

In [8]:
@traceable(name="Summarize Node")
def summarize_all(state: AgentState):
    prompt = f"""
    Explanation:
    {state.explanation}

    Examples:
    {state.examples}

    Create a short summary for revision.
    """
    response = llm.invoke(prompt)

    return {"final_answer": response.content}


### Agent Graph

In [9]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)

workflow.add_node("explain", explain_concept)
workflow.add_node("examples", generate_examples)
workflow.add_node("summary", summarize_all)

workflow.set_entry_point("explain")
workflow.add_edge("explain", "examples")
workflow.add_edge("examples", "summary")
workflow.add_edge("summary", END)

graph = workflow.compile()


### Run Agent

In [10]:
result = graph.invoke(
    AgentState(user_query="What is overfitting in machine learning?")
)

In [11]:
print(result["final_answer"])

### Overfitting: Revision Summary

**What is Overfitting?**
When a machine learning model learns the **training data *too perfectly***, memorizing specific details and noise rather than general patterns.

**Analogy:**
Like an **"overfitting student"** who memorizes *exact answers* to practice questions but fails on a real test with slightly different questions because they didn't understand the underlying concepts.

**Key Characteristics:**
*   **Excellent performance** on the data it was trained on (training data).
*   **Poor performance** on new, unseen data (real-world data).
*   It's **too specialized** in its training data.

**In a Nutshell:**
Your model has **memorized instead of learned**, making it ineffective and useless for real-world predictions on new situations.


In [12]:
print(result["explanation"])

Imagine you're teaching a computer to do something, like recognize cats in pictures or predict house prices.

Here's the simple breakdown of **overfitting**:

---

### The Goal of Machine Learning

The whole point of machine learning is to teach a computer model to learn patterns from existing data (called **training data**) so it can make good predictions or decisions on **new, unseen data**.

Think of it like a student studying for a test.

### The Analogy: The Over-Studious Student

1.  **The Good Student (Ideal Model):**
    *   This student studies the *concepts* and *general rules* from their textbook and practice questions.
    *   They understand *why* things work the way they do.
    *   When they get to the real test, even if the questions are phrased slightly differently or are completely new, they can apply their understanding and do well.

2.  **The Overfitting Student (Overfit Model):**
    *   This student doesn't bother to understand the concepts. Instead, they **memori

In [13]:
print(result["examples"])

Here are 3 real-world examples of overfitting, based on the explanation:

1.  **Medical AI for Disease Detection:**
    *   **Scenario:** An AI model is developed to diagnose a rare skin disease by analyzing images of skin lesions. It's trained on a dataset of thousands of images, but all these images come from a single, highly specialized clinic. This clinic uses a specific type of camera, consistent lighting, and always positions the patient in the exact same way. The AI achieves an astonishing 99.9% accuracy on this training data.
    *   **Overfitting:** The AI hasn't just learned the visual characteristics of the disease; it has also "memorized" the specific lighting conditions, camera artifacts, background elements, and even the subtle angle at which the photos were taken at that particular clinic. It's like the "overfitting student" who memorized the exact phrasing of practice questions.
    *   **Failure on New Data:** When this AI is deployed to a different hospital that uses 